In [22]:
import numpy as np
import pandas as pd
import os
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
print(os.getcwd())
file = '../data/SNP_in_200GENE_chr1.csv'
data = pd.read_csv(file, error_bad_lines=False)

e:\OneDrive\Programming\Python\CVc_in_bio_informatics\bslmm_simulation


C:\Users\Archibald\AppData\Local\Temp\ipykernel_14148\1080821265.py:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv(file, error_bad_lines=False)


In [24]:
data_wo_gene = data.drop('GENE', axis = 1)

In [25]:
# remove duplicated SNP
data_wo_gene.drop_duplicates(inplace=True, ignore_index = True)

In [26]:
# select 10000 SNP
data_selected = data_wo_gene.sample(10000,random_state=0)

In [27]:
# Get Beta
np.random.seed(1)
large_effect = 400
d = 10
beta_large = np.random.normal(0, np.sqrt(large_effect), d)
print(beta_large)

n = len(data_selected)
small_effect = 1
beta_small = np.random.normal(0, np.sqrt(small_effect), n-d)
beta = np.concatenate([beta_large, beta_small])

beta.shape

[ 32.48690727 -12.23512827 -10.56343505 -21.45937244  17.30815259
 -46.03077394  34.89623528 -15.22413802   6.38078192  -4.98740751]


(10000,)

In [28]:
data_selected.head()

,POS,HG00096,HG00097,HG00099,HG00100,HG00101,HG00102,HG00103,HG00104,HG00105,...,NA21128,NA21129,NA21130,NA21133,NA21135,NA21137,NA21141,NA21142,NA21143,NA21144
5322,115618541,2,2,2,0,2,2,2,2,2,...,1,1,1,2,2,0,0,0,0,1
1022,40159031,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7052,175412929,0,1,0,0,0,0,0,0,0,...,1,0,1,2,1,0,0,0,1,1
9786,244831639,1,0,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,1,0,0
8193,183340384,2,1,1,1,2,1,1,2,1,...,1,2,1,1,2,1,2,2,1,1


In [29]:
SNP = data_selected.drop([ 'POS'], axis=1, inplace=False).T
SNP.head()

,5322,1022,7052,9786,8193,6032,159,2293,9742,9972,...,5410,4882,739,7601,5102,586,2989,4444,6567,9220
HG00096,2,0,0,1,2,0,0,1,1,1,...,2,1,2,0,1,2,1,0,1,0
HG00097,2,0,1,0,1,0,0,1,1,1,...,1,0,1,0,0,1,1,0,2,0
HG00099,2,0,0,0,1,2,0,2,0,1,...,1,1,2,0,0,2,1,0,2,0
HG00100,0,0,0,0,1,0,0,2,0,1,...,2,0,2,0,1,1,1,0,2,0
HG00101,2,0,0,0,2,0,2,2,0,1,...,1,1,2,0,0,2,1,0,1,0


In [30]:
# calculate the sigma_e2
temp = SNP.values @ beta
print(temp.shape)
sigma_g2 = np.var(temp, ddof = 1) # get the overall variance
h2 = 0.5 # heritability
sigma_e2 = sigma_g2 * (1 - h2)/ h2
print("The sigma_g^2 is {:.5}, and sigma_e^2 is {:.5}".format(sigma_g2, sigma_e2))

# simulate beta_0 and calculate y
np.random.seed(1)
beta0 = np.random.normal(0, np.sqrt(large_effect))
print(beta0)
print(beta.shape)
# beta_ = np.append(beta, beta0)
beta_ = np.insert(beta, 0, beta0)
print(beta_[:20])
residual = np.random.normal(0, np.sqrt(sigma_e2), len(SNP))
print(SNP.shape)
print(beta_.shape)
Y = SNP.values @ beta + residual + beta0

(2548,)
The sigma_g^2 is 4828.9, and sigma_e^2 is 4828.9
32.48690727326483
(10000,)
[ 3.24869073e+01  3.24869073e+01 -1.22351283e+01 -1.05634350e+01
 -2.14593724e+01  1.73081526e+01 -4.60307739e+01  3.48962353e+01
 -1.52241380e+01  6.38078192e+00 -4.98740751e+00  1.46210794e+00
 -2.06014071e+00 -3.22417204e-01 -3.84054355e-01  1.13376944e+00
 -1.09989127e+00 -1.72428208e-01 -8.77858418e-01  4.22137467e-02]
(2548, 10000)
(10001,)


In [31]:
# get SNPs for simulation data 
SNPs = SNP.copy()
mean_Y = np.mean(Y)
sd_Y = np.std(Y)
SNPs['Y'] = (Y-mean_Y)/sd_Y
SNPs.insert(0, 'bias',1)
SNPs.loc['beta', :]= np.concatenate([beta_,[0]])
path = "../sim_data"
if not os.path.exists(path):
    os.makedirs(path)

SNPs.to_csv(os.path.join(path, 'Simulated_10000_SNPs.csv'))
np.savez(os.path.join(path, 'simulated_parameters_10000_SNPs.npz'), 
sigma_e2 =sigma_e2,sigma_g2 =sigma_g2, h2 = h2, beta = beta_, large_effect_terms = d, mean_Y = mean_Y,sd_Y = sd_Y)

SNPs.head()

,bias,5322,1022,7052,9786,8193,6032,159,2293,9742,...,4882,739,7601,5102,586,2989,4444,6567,9220,Y
HG00096,1.0,2.0,0.0,0.0,1.0,2.0,0.0,0.0,1.0,1.0,...,1.0,2.0,0.0,1.0,2.0,1.0,0.0,1.0,0.0,0.329198
HG00097,1.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,...,0.0,1.0,0.0,0.0,1.0,1.0,0.0,2.0,0.0,-0.415258
HG00099,1.0,2.0,0.0,0.0,0.0,1.0,2.0,0.0,2.0,0.0,...,1.0,2.0,0.0,0.0,2.0,1.0,0.0,2.0,0.0,-2.461198
HG00100,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,...,0.0,2.0,0.0,1.0,1.0,1.0,0.0,2.0,0.0,-0.466995
HG00101,1.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,2.0,0.0,...,1.0,2.0,0.0,0.0,2.0,1.0,0.0,1.0,0.0,-0.004313


In [32]:
data_selected.insert(1, 'major', 'A')
data_selected.insert(2, 'minor', 'G')
data_selected.head()

,POS,major,minor,HG00096,HG00097,HG00099,HG00100,HG00101,HG00102,HG00103,...,NA21128,NA21129,NA21130,NA21133,NA21135,NA21137,NA21141,NA21142,NA21143,NA21144
5322,115618541,A,G,2,2,2,0,2,2,2,...,1,1,1,2,2,0,0,0,0,1
1022,40159031,A,G,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7052,175412929,A,G,0,1,0,0,0,0,0,...,1,0,1,2,1,0,0,0,1,1
9786,244831639,A,G,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
8193,183340384,A,G,2,1,1,1,2,1,1,...,1,2,1,1,2,1,2,2,1,1


In [33]:
data_selected_resetindex = data_selected.reset_index(drop=True, inplace=False)

In [34]:
data_selected_resetindex

,POS,major,minor,HG00096,HG00097,HG00099,HG00100,HG00101,HG00102,HG00103,...,NA21128,NA21129,NA21130,NA21133,NA21135,NA21137,NA21141,NA21142,NA21143,NA21144
0,115618541,A,G,2,2,2,0,2,2,2,...,1,1,1,2,2,0,0,0,0,1
1,40159031,A,G,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,175412929,A,G,0,1,0,0,0,0,0,...,1,0,1,2,1,0,0,0,1,1
3,244831639,A,G,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
4,183340384,A,G,2,1,1,1,2,1,1,...,1,2,1,1,2,1,2,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,29554965,A,G,2,1,2,1,2,2,2,...,2,2,2,2,2,2,2,2,2,2
9996,68182033,A,G,1,1,1,1,1,0,1,...,0,1,0,1,1,0,0,0,0,0
9997,78748525,A,G,0,0,0,0,0,1,1,...,0,0,1,0,0,0,1,0,0,0
9998,173390597,A,G,1,2,2,2,1,1,0,...,1,1,2,2,0,1,0,0,2,1


In [35]:
bimbam_path = '../bimbam_data/'
if not os.path.exists(bimbam_path):
    os.mkdir(bimbam_path)
data_selected_resetindex.to_csv(
    os.path.join(bimbam_path,'bimbam_10000_full_false_major_minor.txt'), sep = '\t', index = False)

### Load the bimbam data

In [40]:
import pandas as pd
import numpy as np
import os
bimbam_path = '../bimbam_data/'

data_selected_resetindex = pd.read_csv(os.path.join(bimbam_path,'bimbam_10000_full_false_major_minor.txt'), sep = '\t',index_col=None)

In [41]:
data_selected_resetindex.head()

,POS,major,minor,HG00096,HG00097,HG00099,HG00100,HG00101,HG00102,HG00103,...,NA21128,NA21129,NA21130,NA21133,NA21135,NA21137,NA21141,NA21142,NA21143,NA21144
0,115618541,A,G,2,2,2,0,2,2,2,...,1,1,1,2,2,0,0,0,0,1
1,40159031,A,G,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,175412929,A,G,0,1,0,0,0,0,0,...,1,0,1,2,1,0,0,0,1,1
3,244831639,A,G,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
4,183340384,A,G,2,1,1,1,2,1,1,...,1,2,1,1,2,1,2,2,1,1


In [24]:
from sklearn.model_selection import train_test_split as split
name = data_selected_resetindex.columns[3:]
sample_size = len(name)
train_size = int(0.8 * len(name))
test_size = sample_size - train_size
train_idx, test_idx = split(np.arange(len(name)), test_size=0.2, random_state=123)

In [33]:
from bimbam import data_spliter
data_spliter

bimbam_path = '../bimbam_data/'

(bimbam_train, pheno_tr), (bimbam_test, pheno_te),(bimbam_full, pheno_full) =\
 data_spliter(data_selected_resetindex ,SNPs['Y'],train_idx.tolist(),test_idx.tolist())

Training set has 2035 samples
Test set has 507 samples


In [40]:
bimbam_train.to_csv(bimbam_path + 'bimbam_train', header=False, index=False)
bimbam_test.to_csv(bimbam_path + 'bimbam_test', header=False, index=False)
bimbam_full.to_csv(bimbam_path + 'bimbam_full', header=False, index=False)
np.savetxt(bimbam_path +'pheno_tr', pheno_tr)
np.savetxt(bimbam_path +'pheno_te', pheno_te)
np.savetxt(bimbam_path +'pheno_full', pheno_full, fmt="%s")

In [30]:
df = data_selected_resetindex.iloc[:, :-1]
df.to_csv("../bimbam_data/bimbam_train",index = False, header = False)

In [29]:
np.savetxt(os.path.join(bimbam_path,'effect_size'), beta)

## create relatedness matrix

In [2]:
data_tr = pd.read_csv('../bimbam_data/bimbam_train')
data_te = pd.read_csv('../bimbam_data/bimbam_test')

In [5]:
tr = data_tr.iloc[:,3:].to_numpy()
p = tr.shape[0]
K_tr = 1/p * tr.T @ tr

In [8]:
np.savetxt('../bimbam_data/K_tr', K_tr )